In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Load your data
df = pd.read_csv("jobs_in_data.csv")

# Define features and target variable
X_text = df["job_title"]  # Assuming "job_title" contains the text data
X_numerical = df.drop(columns=["job_title", "salary"])  # Exclude non-numeric columns
y = df["salary"]

# Tokenize text data
max_sequence_length = 50  # Define the maximum sequence length for padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_text)
X_text_seq = tokenizer.texts_to_sequences(X_text)
X_text_padded = pad_sequences(X_text_seq, maxlen=max_sequence_length)

# Scale numerical data
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Concatenate text and numerical data
X = np.concatenate([X_text_padded, X_numerical_scaled], axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model
input_layer = Input(shape=(X_train.shape[1],))
lstm_layer = LSTM(64)(input_layer)
output_layer = Dense(1, activation='linear')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", np.sqrt(mse))


ValueError: could not convert string to float: 'Data Engineering'